# RNA-seqの解析練習

## チャ Camellia sinensis のモチ病（Blister Blight）
基本情報
- チャ: ツバキ科ツバキ属の木本植物
- チャもち病菌: Exobasidium vexans; 担子菌類
- チャもち病発生地域: 茶の生産地域（スリランカ、インド、インドネシア、日本など）で発生する
- 被害: 茶の品質低下、収量低下を引き起こす

## RNA-seqサンプル
_Jayaswall, K., Mahajan, P., Singh, G., Parmar, R., Seth, R., Raina, A., ... & Sharma, R. K. (2016). Transcriptome analysis reveals candidate genes involved in blister blight defense in tea (Camellia sinensis (L) Kuntze). Scientific reports, 6, 30412._

- 抵抗性品種 SA6
- 感受性品種 Kangra Asha
- 病原菌接種14日後のRNA-seqサンプル  
    病原菌が（感受性品種の）植物細胞に侵入して植物から栄養を搾取する吸器を伸ばす時期

## データ概要
### 発現比較データ（./tea_set/S3.dataset.txt）


In [2]:
import pandas as pd
df = pd.read_csv('./tea_set/S3.dataset.txt', sep='\t', header=0, index_col=0)
df.head()

#================================
#    - 遺伝子名
#    - S3_RG: 抵抗性品種の遺伝子発現量（TMM正規化後）
#    - S3_SG: 感受性品種の遺伝子発現量（TMM正規化後）
#    - logFC: log2(抵抗性品種/感受性品種)の値  
#        正の値なら抵抗性品種の発現量が高い  
#        負の値なら感受性品種の発現量が高い  
#    - logCPM: log2((抵抗性品種 + 感受性品種)/2)の値  
#        この値が大きいほど、発現量の大きい遺伝子である指標
#    - PValue: Fisher's exact testによる有意確率（p-value）  
#    - FDR: Benjamini-Hochberg法によるFalse Discovery Rate（FDR）  
#        通常、この値が0.05より小さい場合、有意である（発現量に差がある）とみなす。

,S3_RG,S3_SG,logFC,logCPM,PValue,FDR
TRINITY_DN7059_c0_g2,740.059,0.108,12.590532,8.799814,2.650976e-26,5.956743e-22
TRINITY_DN7059_c0_g1,1268.262,0.637,10.689974,6.902134,1.306793e-20,1.468182e-16
TRINITY_DN6936_c2_g1,232.299,0.143,10.530966,6.743582,3.896130e-20,2.918201e-16
TRINITY_DN8566_c3_g2,112.268,0.000,13.314056,6.349821,5.356733e-20,3.009145e-16
TRINITY_DN7894_c0_g1,79.082,0.000,13.268257,6.304171,7.344590e-20,3.300659e-16


---

### 発現変動遺伝子のBLAST結果リスト（./tea_set/S3.DEGs_BLAST.txt） 

In [3]:
import pandas as pd
df_BLAST = pd.read_csv('./tea_set/S3.DEGs_BLAST.txt', sep='\t', header=0, index_col=0)
df_BLAST.head()

#================================
#    - gene_id: 遺伝子名
#    - BLASTX_swissprot:  
#        UniProtKB/Swiss-Protデータベースに対してBLASTXをして、ベストスコアとして得られた情報

,BLASTX_swissprot
TRINITY_DN7059_c0_g2,"RD22_ARATH^RD22_ARATH^Q:965-288,H:161-390^49.7..."
TRINITY_DN7059_c0_g1,.
TRINITY_DN6936_c2_g1,.
TRINITY_DN8566_c3_g2,.
TRINITY_DN7894_c0_g1,"NATT3_THANI^NATT3_THANI^Q:911-1435,H:181-354^2..."


---
### 発現変動遺伝子の塩基配列（./tea_set/S3.DEGs.fasta）

In [5]:
import pandas as pd
FASTA = open('./tea_set/S3.DEGs.fasta', 'r')

# データフレームに変換
seqs = { 'id':[], 'seq':[] }
sw = 0
for line in FASTA:
    sw += 1
    line = line.rstrip()
    if sw % 2 == 1:
        seqs['id'].append(line[1:])
    else:
        seqs['seq'].append(line)
df_FASTA = pd.DataFrame(seqs).set_index(['id'])
df_FASTA.head()

#================================
#    - id : 遺伝子名
#    - seq: 塩基配列

,seq
id,
TRINITY_DN7059_c0_g2_i1,TTTTTTTAAGCATATAAAGTTGGTTGAAGTTATTAATTAATATAAA...
TRINITY_DN7059_c0_g1_i1,GCATCCTTATCCCCTCTGCCCACGTTGTAAAGAATCGCTCTTGCAT...
TRINITY_DN6936_c2_g1_i1,TCGGCGTTGGGTGTGTTCGACCATAGGAGTGGAGAAGTGGAGTCGG...
TRINITY_DN8566_c3_g2_i1,GCCAAGTGATATGGTGATTCCGGAGTATTTACGCCATTATGTTAGG...
TRINITY_DN7894_c0_g1_i1,AGTGAGAGAGAGAGAGAGAGAGAGAGAGAGAATGGCACTGCCAAGG...


---
## 今回の解析内容
- 発現量を比較した遺伝子の総数は？


- 有意な発現（FDR < 0.05）とみなせる遺伝子数は？


- 発現に差がある遺伝子のうち、S3_RG（抵抗性品種）で高発現の遺伝子数は？


- 発現に差がある遺伝子のうち、S3_SG（感受性品種）で高発現の遺伝子数は？


- S3_RG（抵抗性品種）の高発現遺伝子には、病原菌応答関連遺伝子が含まれているか？


- S3_SG（感受性品種）の高発現遺伝子には、病原菌応答関連遺伝子が含まれているか？  
    

- もち病への抵抗性-感受性を決めている遺伝子（候補）は何か？


---

病原菌抵抗性関連遺伝子のキーワード
- resistance
- pathogen
- LRR (Leucine-rich repeat)
- WRKY
- NAC domain
- chitinase
- peroxidase
- pectinesterase inhibitor
- jasmonate-induced protein

データベース/ツール
- NCBI BLAST 配列相同性検索: https://blast.ncbi.nlm.nih.gov/Blast.cgi
- Uniprot 遺伝子/タンパク質データベース: https://www.uniprot.org/
- Google scholar 論文検索: https://scholar.google.co.jp/

その他解析ツール
- BLAST検索プログラム

```python
import remote_ncbi as rn
rn.ncbi_blast(BLASTの種類, 塩基配列)

# BLASTの種類: 'blastn', 'blastx', など
```
